# Part 1: Data Preprocessing

use Markdown cells to describe what is going on

In [3]:
import requests 
import bs4
import pandas as pd
import re
from math import sin, cos, sqrt, atan2, radians
from io import StringIO
import pandas as pd
import glob
import os

# Part 1: Data Preprocessing

## Uber Dataset

In [4]:
df_uber=pd.read_csv("uber_rides_sample.csv")

In [5]:
def distance(row):
    R = 6373.0
    
    lon1 = radians(row["pickup_longitude"])
    lat1 = radians(row["pickup_latitude"])
    
    lon2 = radians(row["dropoff_longitude"])
    lat2 = radians(row["dropoff_latitude"])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [6]:
def uber_data_cleaning(df_uber):
    df_uber.dropna()
    df_uber.drop("Unnamed: 0" , axis=1, inplace=True)
    
    pick_long_check=(df_uber["pickup_longitude"]>= -74.242330) & (df_uber["pickup_longitude"]<= -73.717047)
    drop_long_check=(df_uber["dropoff_longitude"]>= -74.242330) & (df_uber["dropoff_longitude"]<= -73.717047)
    pick_latt_check=(df_uber["pickup_latitude"]>= 40.560445) & (df_uber["pickup_latitude"]<= 40.908524)
    drop_latt_check=(df_uber["dropoff_latitude"]>= 40.560445) & (df_uber["dropoff_latitude"]<= 40.908524)

    df_uber=df_uber[pick_long_check & drop_long_check & pick_latt_check & drop_latt_check]
    
    df_uber['pickup_datetime'] = pd.to_datetime(df_uber['pickup_datetime'])
    df_uber['pickup_datetime']=df_uber['pickup_datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    df_uber['distance'] = df_uber.apply(distance, axis=1)
    
    return df_uber

In [7]:
df_uber=uber_data_cleaning(df_uber)

<ipython-input-6-de374cc63ef3>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uber['pickup_datetime'] = pd.to_datetime(df_uber['pickup_datetime'])
<ipython-input-6-de374cc63ef3>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uber['pickup_datetime']=df_uber['pickup_datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
<ipython-input-6-de374cc63ef3>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [8]:
df_uber

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
0,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06,-73.999817,40.738354,-73.999512,40.723217,1,1.683851
1,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56,-73.994355,40.728225,-73.994710,40.750325,1,2.458361
2,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00,-74.005043,40.740770,-73.962565,40.772647,1,5.037958
3,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21,-73.976124,40.790844,-73.965316,40.803349,3,1.662205
4,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00,-73.925023,40.744085,-73.973082,40.761247,5,4.476855
...,...,...,...,...,...,...,...,...,...
199995,2012-10-28 10:49:00.00000053,3.0,2012-10-28 10:49:00,-73.987042,40.739367,-73.986525,40.740297,1,0.112245
199996,2014-03-14 01:09:00.0000008,7.5,2014-03-14 01:09:00,-73.984722,40.736837,-74.006672,40.739620,1,1.875639
199997,2009-06-29 00:42:00.00000078,30.9,2009-06-29 00:42:00,-73.986017,40.756487,-73.858957,40.692588,2,12.854353
199998,2015-05-20 14:56:25.0000004,14.5,2015-05-20 14:56:25,-73.997124,40.725452,-73.983215,40.695415,1,3.540827


In [ ]:
# df_uber.to_csv('uber_cleaned.csv',index=False)

## Yellow Taxi Dataset

In [ ]:
def link_parser(): 
    # Scrapping out link for each of yellow trip data
    
    response = requests.get('https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page')
    soup = bs4.BeautifulSoup(response.content, 'html.parser')
    parse = str(soup.find_all("li")).split()
    pattern = re.compile(r"href=\"https:\/\/s3\.amazonaws\.com\/nyc\-tlc\/trip\+data\/yellow\_tripdata\_20[0-1]\d")
    newlist = list(filter(pattern.match, parse))
    newlist = newlist[42:]
    linklist=[]
    for item in newlist:
        linklist.append(item.split('href="')[1][:-1])
    return linklist


In [ ]:
linklist=link_parser()

In [ ]:
def yellow_taxi_dataset(linklist):
    result=pd.DataFrame()
    #Data collection of 2015's yellow taxi data
    for link in linklist[:12]:
        req = requests.get(link)
        url_content = req.content
    
        s=str(url_content,'utf-8')
        data = StringIO(s) 
        df=pd.read_csv(data, error_bad_lines=False)
        df.drop("improvement_surcharge" , axis=1, inplace=True)
        df = df.iloc[: , :-1]
        df.columns=['vendor_id','pickup_datetime','dropoff_datetime','passenger_count','trip_distance','pickup_longitude',
                'pickup_latitude','rate_code','store_and_fwd_flag','dropoff_longitude','dropoff_latitude','payment_type',
                'fare_amount','surcharge','mta_tax','tip_amount','tolls_amount','total_amount']
        df = df.sample(n=3000)
    
        result=result.append(df)
    #Data collection of years from 2009 to 2014    
    for link in linklist[12:]:
        req = requests.get(link)
        url_content = req.content
    
        s=str(url_content,'utf-8')
        data = StringIO(s) 
        df=pd.read_csv(data, error_bad_lines=False)
        df.columns=['vendor_id','pickup_datetime','dropoff_datetime','passenger_count','trip_distance','pickup_longitude',
                'pickup_latitude','rate_code','store_and_fwd_flag','dropoff_longitude','dropoff_latitude','payment_type',
                'fare_amount','surcharge','mta_tax','tip_amount','tolls_amount','total_amount']
        df = df.sample(n=3000)
    
        result=result.append(df)
        
    return result
    

In [ ]:
df_taxi=yellow_taxi_dataset(linklist)

In [9]:
df_taxi=pd.read_csv("Yellow_Taxi_Sample.csv")

/Users/lisakuai/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,8,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
def clean_yellowtaxi(df_taxi):
    
    #location filter
    pick_long_check=(df_taxi["pickup_longitude"]>= -74.242330) & (df_taxi["pickup_longitude"]<= -73.717047)
    drop_long_check=(df_taxi["dropoff_longitude"]>= -74.242330) & (df_taxi["dropoff_longitude"]<= -73.717047)
    pick_latt_check=(df_taxi["pickup_latitude"]>= 40.560445) & (df_taxi["pickup_latitude"]<= 40.908524)
    drop_latt_check=(df_taxi["dropoff_latitude"]>= 40.560445) & (df_taxi["dropoff_latitude"]<= 40.908524)

    df_taxi=df_taxi[pick_long_check & drop_long_check & pick_latt_check & drop_latt_check]
    
    #drop columns with too many NAs
    na_bar = len(df_taxi) * .8
    df_taxi = df_taxi.dropna(thresh=na_bar, axis=1)
    
    #add distance column
    df_taxi['distance'] = df_taxi.apply(distance, axis=1)
    
    #drop trip_distance col
    df_taxi = df_taxi.drop(['trip_distance'], axis = 1)

    
    return df_taxi

In [11]:
#df_taxi=pd.read_csv("Yellow_Taxi_Sample.csv")
df_taxi=clean_yellowtaxi(df_taxi)
df_taxi

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,distance
0,1,2015-01-24 14:58:38,2015-01-24 15:02:19,1,-73.956810,40.781162,-73.955429,40.773361,1,5.0,0.0,0.5,1.15,0.0,6.95,0.875471
1,1,2015-01-08 15:39:52,2015-01-08 15:44:22,1,-73.975601,40.752163,-73.981140,40.744507,2,4.5,0.0,0.5,0.00,0.0,5.30,0.971107
2,2,2015-01-07 15:21:40,2015-01-07 15:29:07,1,-73.952255,40.777321,-73.959785,40.766941,2,6.5,0.0,0.5,0.00,0.0,7.30,1.317316
3,2,2015-01-10 18:19:43,2015-01-10 18:26:58,1,-73.995934,40.732300,-73.997925,40.736084,2,6.0,0.0,0.5,0.00,0.0,6.80,0.453140
4,2,2015-01-24 12:16:16,2015-01-24 12:30:55,5,-73.978249,40.752361,-74.005623,40.715229,2,13.5,0.0,0.5,0.00,0.0,14.30,4.730957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251995,CMT,2009-12-13 09:12:35,2009-12-13 09:14:35,1,-73.990913,40.670626,-73.991142,40.669925,Cash,3.7,0.0,0.5,0.00,0.0,4.20,0.080330
251996,VTS,2009-12-24 14:19:00,2009-12-24 14:37:00,2,-73.975545,40.728643,-73.951623,40.783325,CASH,13.7,0.0,0.5,0.00,0.0,14.20,6.407539
251997,CMT,2009-12-27 21:03:51,2009-12-27 21:22:37,1,-73.999803,40.726753,-73.993088,40.665041,Cash,19.7,0.5,0.5,0.00,0.0,20.70,6.887535
251998,CMT,2009-12-12 20:25:55,2009-12-12 20:41:44,3,-73.997882,40.751496,-73.993273,40.722716,Cash,10.5,0.5,0.5,0.00,0.0,11.50,3.224676


In [12]:
#Test Case for Distance Function
def distance_test():
    
    distance_test=round(distance(df_taxi.iloc[[0]]),2)   
    assert distance_test== 0.88

In [13]:
distance_test()

## Weather Dataset

In [14]:
def concact_weather_data():
    # setting the path for joining multiple files
    files = os.path.join("/Users/nat/Desktop/tfa/proj", "*weather.csv") #directory change needed

    # list of merged files returned
    files = glob.glob(files)

    # joining files with concat and store csv
    df_weather = pd.concat(map(pd.read_csv, files), ignore_index=True)
    #df_weather.to_csv('weather_all.csv',index=False)  
    
    return df_weather

In [15]:
df_weather=concact_weather_data()

ValueError: No objects to concatenate

In [16]:
df_weather=pd.read_csv("weather_all.csv")

/Users/lisakuai/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7,8,9,10,13,17,18,19,40,41,42,61,65,78,88,89,90,91,92,93,94,95,96,97,98,99) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
def clean_weather(df_weather):
    #choose only useful cols
    df_weather_cleaned = df_weather[df_weather.filter(regex='DATE|speed|Speed|Precipitation|precipitation').columns[:8]]
    
    df_weather_hourly = df_weather_cleaned[df_weather_cleaned.filter(regex='DATE|hourly|Hourly').columns[:]]
    df_weather_daily = df_weather_cleaned[df_weather_cleaned.filter(regex='DATE|daily|Daily').columns[:]]
    
    return df_weather_hourly,df_weather_daily

In [18]:
df_weather_hourly,df_weather_daily=clean_weather(df_weather)

# Part 2: Storing Data

#Use SQLAlchemy to create a SQLite
#explanation needs

In [19]:
#Question:
# 1. Is it correct to store 4 tables into final_proj.db?
# 2. daily weather data; too many missing values
# 3. how to save as schema.sqlp file? we only have .db file now

In [20]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import validates

In [37]:
def database():
    
    engine = create_engine(f"sqlite:///final_project.db", echo=True)
    Base = declarative_base()
    
    # Convert csv file into sql & insert into data.db
    df_uber.to_sql('Uber_trips', con=engine, index=True, index_label='id', if_exists='replace')
    df_taxi.to_sql('Yellow_Taxi_trips', con=engine, index=True, index_label='id', if_exists='replace')
    df_weather_daily.to_sql('Daily_Weather_Information', con=engine, index=True, index_label='id', if_exists='replace')
    df_weather_hourly.to_sql('Hourly_Weather_Information', con=engine, index=True, index_label='id', if_exists='replace')

In [38]:
database()

2022-04-22 01:20:13,053 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Uber_trips")
2022-04-22 01:20:13,055 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-22 01:20:13,056 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Uber_trips")
2022-04-22 01:20:13,056 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-22 01:20:13,058 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-22 01:20:13,058 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Uber_trips" (
	id BIGINT, 
	"key" TEXT, 
	fare_amount FLOAT, 
	pickup_datetime TEXT, 
	pickup_longitude FLOAT, 
	pickup_latitude FLOAT, 
	dropoff_longitude FLOAT, 
	dropoff_latitude FLOAT, 
	passenger_count BIGINT, 
	distance FLOAT
)


2022-04-22 01:20:13,059 INFO sqlalchemy.engine.Engine [no key 0.00029s] ()
2022-04-22 01:20:13,060 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Uber_trips_id" ON "Uber_trips" (id)
2022-04-22 01:20:13,061 INFO sqlalchemy.engine.Engine [no key 0.00039s] ()
2022-04-22 01:20:13,062 INFO sqlalchemy.engine.Engi

2022-04-22 01:20:18,527 INFO sqlalchemy.engine.Engine [generated in 0.24651s] ((0, '2012-01-01T00:51:00', None, None, None, None), (1, '2012-01-01T01:51:00', None, None, None, None), (2, '2012-01-01T02:51:00', None, None, None, None), (3, '2012-01-01T03:51:00', None, None, None, None), (4, '2012-01-01T04:51:00', None, None, None, None), (5, '2012-01-01T05:15:00', None, None, None, None), (6, '2012-01-01T05:35:00', None, None, None, None), (7, '2012-01-01T05:51:00', None, None, None, None)  ... displaying 10 of 77972 total bound parameter sets ...  (77970, '2010-12-31T22:51:00', None, None, None, None), (77971, '2010-12-31T23:51:00', None, None, None, None))
2022-04-22 01:20:18,754 INFO sqlalchemy.engine.Engine COMMIT
2022-04-22 01:20:18,759 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-04-22 01:20:18,759 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-22 01:20:18,772 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Hourly_Wea

# Part 3: Understanding Sata

In [23]:
import sqlite3
connection = sqlite3.connect("final_project.db")
connection

1.	For 01-2009 through 06-2015, what hour of the day was the most popular to take a Yellow Taxi? The result should have 24 bins.

In [24]:
with connection:
    Q1 = connection.execute(
        """SELECT strftime('%H', y.pickup_datetime) as Hour, count(*) as Counts
                                From Yellow_Taxi_trips y
                                where date(y.pickup_datetime)>="2009-01-01" and date(y.pickup_datetime)<="2015-06-30"
                                group by Hour
                                order by Counts desc""")

In [25]:
for row in Q1:
    print(row)

('19', 14246)
('18', 13868)
('20', 13576)
('21', 13047)
('22', 12742)
('14', 11446)
('17', 11335)
('23', 11295)
('12', 11085)
('15', 11070)
('13', 11057)
('09', 10625)
('08', 10471)
('11', 10438)
('10', 10356)
('16', 9615)
('00', 9141)
('07', 8282)
('01', 6766)
('02', 5013)
('06', 4703)
('03', 3685)
('04', 2713)
('05', 2224)


2.	For the same time frame, what day of the week was the most popular to take an Uber? The result should have 7 bins.

In [26]:
with connection:
    Q2 = connection.execute(
        """SELECT strftime('%w', u.pickup_datetime) as weekofday, count(*) as Counts
                                From Uber_trips u
                                where date(u.pickup_datetime)>="2009-01-01" and date(u.pickup_datetime)<="2015-06-30"
                                group by weekofday
                                order by Counts desc""")

In [27]:
for item in Q2:
    print(item)

('5', 30166)
('6', 29599)
('4', 29338)
('3', 28328)
('2', 27526)
('0', 25834)
('1', 24681)


3.	What is the 95% percentile of distance traveled for all hired trips during July 2013?

In [42]:
with connection:
    Q3 = connection.execute(
        """ with newtable as 
                (SELECT distance from Uber_trips
                 Where date(pickup_datetime)<="2013-07-31" and date(pickup_datetime)>="2013-07-1"
            
                 Union all
            
                SELECT distance from Yellow_Taxi_trips
                Where date(pickup_datetime)<="2013-07-31" and date(pickup_datetime)>="2013-07-1")
            
            Select distance as "95% distance"
            FROM newtable
            ORDER BY distance ASC
            LIMIT 1
            OFFSET (SELECT
             COUNT(*)
            FROM newtable
            ) * 95 / 100 - 1
            
            """)

In [43]:
for item in Q3:
    print(item)

(10.430214496267116,)
